In [1]:
from autocnet_server.db import model
import json

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Integer, Float, JSON, ForeignKey, Boolean, LargeBinary, UniqueConstraint
# from sqlalchemy.dialects.postgresql import ARRAY, JSONB
from sqlalchemy.orm import relationship, backref
from geoalchemy2 import Geometry
from geoalchemy2.shape import to_shape
from sqlalchemy import Sequence
import json
import pandas as pd
from sqlalchemy.types import (
    Integer,
    String,
    TypeDecorator,
    Numeric,
    )

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Sequence



Base = declarative_base()

# Custom class for Array
class ArrayType(TypeDecorator):
    """ Sqlite-like does not support arrays.
        Let's use a custom type decorator.

        See http://docs.sqlalchemy.org/en/latest/core/types.html#sqlalchemy.types.TypeDecorator
    """
    
    impl = String

    def process_bind_param(self, value, dialect):
        return json.dumps(value)

    def process_result_value(self, value, dialect):
        return json.loads(value)

    def copy(self):
        return ArrayType(self.impl.length)

# Custom class for JSON
class Json(TypeDecorator):

    @property
    def python_type(self):
        return object

    impl = String

    def process_bind_param(self, value, dialect):
        return json.dumps(value)

    def process_literal_param(self, value, dialect):
        return value

    def process_result_value(self, value, dialect):
        try:
            return json.loads(value)
        except (ValueError, TypeError):
            return None

class Cameras(Base):
    __tablename__ = 'cameras'
    id = Column(Integer, primary_key=True, autoincrement=True)
    image_id = Column(Integer, ForeignKey("edges.id", ondelete="CASCADE"), unique=True)
    camera = Column(Json())
    
class Element(Base):
    __tablename__ = 'elements'

    id = Column(Integer(),
                Sequence('element_id_seq'),
                primary_key = True)
    # ...
    myarray = Column(ArrayType())
    mycolumn = Column(Json())
    
class Edges(Base):
    __tablename__ = 'edges'
    id = Column(Integer, primary_key=True, autoincrement=True)
    source = Column(Integer)
    destination = Column(Integer)
    ring = Column(ArrayType())
    fundamental = Column(ArrayType())
    active = Column(Boolean)
    masks = Column(Json())

/usgs/apps/anaconda/envs/autocnet/lib/python3.6/site-packages/autocnet_server/__init__.py:30: UserWarning: No autocnet_config environment variable set. Defaulting to an en empty configuration.
  warnings.warn('No autocnet_config environment variable set. Defaulting to an en empty configuration.')


In [2]:
engine = create_engine('sqlite:///:memory:', echo=True)

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)

session = Session()

2018-09-14 15:33:09,627 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-09-14 15:33:09,628 INFO sqlalchemy.engine.base.Engine ()
2018-09-14 15:33:09,630 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-09-14 15:33:09,631 INFO sqlalchemy.engine.base.Engine ()
2018-09-14 15:33:09,634 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("cameras")
2018-09-14 15:33:09,635 INFO sqlalchemy.engine.base.Engine ()
2018-09-14 15:33:09,637 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("elements")
2018-09-14 15:33:09,638 INFO sqlalchemy.engine.base.Engine ()
2018-09-14 15:33:09,640 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("edges")
2018-09-14 15:33:09,641 INFO sqlalchemy.engine.base.Engine ()
2018-09-14 15:33:09,644 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE elements (
	id INTEGER NOT NULL, 
	myarray VARCHAR, 
	mycolumn VARCHAR, 
	PRIMARY KEY (id)
)


2018-09-14 15:33:09,6

In [3]:
# Creates a df out of dict and dumps to json
data = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data)
df_json = df.to_json(orient='index')

# creates a test of an array and json
test_data = Element(id=69, myarray =[1, 2, 3, 4, 5], mycolumn=df_json)

# Adds that test
session.add(test_data)

# Query the db for the test_data values
our_user = session.query(Element).filter_by(id=69).first()

# Output both values
our_user.myarray
our_user.mycolumn

2018-09-14 15:33:09,713 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-09-14 15:33:09,716 INFO sqlalchemy.engine.base.Engine INSERT INTO elements (id, myarray, mycolumn) VALUES (?, ?, ?)
2018-09-14 15:33:09,718 INFO sqlalchemy.engine.base.Engine (69, '[1, 2, 3, 4, 5]', '"{\\"0\\":{\\"col1\\":1,\\"col2\\":3},\\"1\\":{\\"col1\\":2,\\"col2\\":4}}"')
2018-09-14 15:33:09,722 INFO sqlalchemy.engine.base.Engine SELECT elements.id AS elements_id, elements.myarray AS elements_myarray, elements.mycolumn AS elements_mycolumn 
FROM elements 
WHERE elements.id = ?
 LIMIT ? OFFSET ?
2018-09-14 15:33:09,723 INFO sqlalchemy.engine.base.Engine (69, 1, 0)


'{"0":{"col1":1,"col2":3},"1":{"col1":2,"col2":4}}'